# Core Character Store System

> Character stores and syllable composition for Geez keyboards

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from typing import Dict, List, Optional, Tuple
from enum import Enum
from dataclasses import dataclass
import json
from pathlib import Path

## Vowel Order System

In Geez script, each consonant has 7 forms corresponding to different vowel sounds. This follows the traditional order:

1. **ግዕዝ** (Geez) - base form (ä sound)
2. **ካዕብ** (Kaeb) - u sound 
3. **ሣልስ** (Salis) - i sound
4. **ራብዕ** (Rabe) - a sound
5. **ኃምስ** (Hams) - e sound
6. **ሳድስ** (Sades) - ə sound (represented by ')
7. **ሳብዕ** (Sabe) - o sound

In [ ]:
#| export
class VowelOrder(Enum):
    """Traditional Geez vowel order system."""
    GEEZ = (0, "ግዕዝ", "", "ä")      # Base form
    KAEB = (1, "ካዕብ", "u", "u")     # Second order
    SALIS = (2, "ሣልስ", "i", "i")    # Third order  
    RABE = (3, "ራብዕ", "a", "a")     # Fourth order
    HAMS = (4, "ኃምስ", "e", "e")     # Fifth order
    SADES = (5, "ሳድስ", "'", "ə")    # Sixth order
    SABE = (6, "ሳብዕ", "o", "o")     # Seventh order
    
    def __init__(self, index, geez_name, latin_key, ipa):
        self.index = index
        self.geez_name = geez_name
        self.latin_key = latin_key
        self.ipa = ipa
    
    @classmethod
    def from_latin_key(cls, key: str) -> Optional['VowelOrder']:
        """Get vowel order from Latin key."""
        for vowel in cls:
            if vowel.latin_key == key:
                return vowel
        return None
    
    @classmethod
    def from_index(cls, index: int) -> Optional['VowelOrder']:
        """Get vowel order from index."""
        for vowel in cls:
            if vowel.index == index:
                return vowel
        return None

In [ ]:
# Test vowel order system
test_eq(VowelOrder.from_latin_key('a'), VowelOrder.RABE)
test_eq(VowelOrder.from_latin_key('e'), VowelOrder.HAMS)
test_eq(VowelOrder.from_index(0), VowelOrder.GEEZ)
test_eq(VowelOrder.RABE.index, 3)
test_eq(VowelOrder.HAMS.geez_name, "ኃምስ")

## Character Families

Each consonant in Geez script forms a family of 7 characters following the vowel order system. For example, the ሀ family:

| Order | Character | Latin | Sound |
|-------|-----------|-------| ------|
| ግዕዝ    | ሀ         | h     | hä    |
| ካዕብ    | ሁ         | hu    | hu    |
| ሣልስ    | ሂ         | hi    | hi    |
| ራብዕ    | ሃ         | ha    | ha    |
| ኃምስ    | ሄ         | he    | he    |
| ሳድስ    | ህ         | h'    | hə    |
| ሳብዕ    | ሆ         | ho    | ho    |

In [ ]:
#| export
@dataclass
class CharacterFamily:
    """Represents a family of Geez characters with vowel variations."""
    base_key: str                    # Latin key (e.g., 'h', 's', 'l')
    characters: List[str]           # 7 characters in vowel order
    name: str                       # Human readable name
    unicode_range: Tuple[int, int]  # Unicode start and end points
    
    def __post_init__(self):
        if len(self.characters) != 7:
            raise ValueError(f"Character family must have exactly 7 characters, got {len(self.characters)}")
    
    def get_character(self, vowel_order: VowelOrder) -> str:
        """Get character for specific vowel order."""
        return self.characters[vowel_order.index]
    
    def get_character_by_index(self, index: int) -> Optional[str]:
        """Get character by vowel index (0-6)."""
        if 0 <= index < 7:
            return self.characters[index]
        return None
    
    def get_character_by_latin_key(self, vowel_key: str) -> Optional[str]:
        """Get character by Latin vowel key."""
        vowel_order = VowelOrder.from_latin_key(vowel_key)
        if vowel_order:
            return self.get_character(vowel_order)
        return None

In [ ]:
# Test character family
h_family = CharacterFamily(
    base_key='h',
    characters=['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ'],
    name='ሀ family (h)',
    unicode_range=(0x1200, 0x1206)
)

test_eq(h_family.get_character(VowelOrder.RABE), 'ሃ')
test_eq(h_family.get_character_by_index(4), 'ሄ')
test_eq(h_family.get_character_by_latin_key('e'), 'ሄ')
test_eq(h_family.get_character_by_latin_key(''), 'ሀ')

## Character Store

The `CharacterStore` class manages all character families and provides efficient lookup functionality.

In [ ]:
#| export
class CharacterStore:
    """Manages character families and provides lookup functionality."""
    
    def __init__(self):
        self.families: Dict[str, CharacterFamily] = {}
        self._character_to_family: Dict[str, str] = {}  # Reverse lookup
        self._initialize_default_families()
    
    def _initialize_default_families(self):
        """Initialize with standard Geez character families."""
        default_families = [
            # Basic consonants
            CharacterFamily('h', ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ'], 'ሀ family (h)', (0x1200, 0x1206)),
            CharacterFamily('l', ['ለ', 'ሉ', 'ሊ', 'ላ', 'ሌ', 'ል', 'ሎ'], 'ለ family (l)', (0x1208, 0x120E)),
            CharacterFamily('hh', ['ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሔ', 'ሕ', 'ሖ'], 'ሐ family (hh)', (0x1210, 0x1216)),
            CharacterFamily('m', ['መ', 'ሙ', 'ሚ', 'ማ', 'ሜ', 'ም', 'ሞ'], 'መ family (m)', (0x1218, 0x121E)),
            CharacterFamily('sz', ['ሠ', 'ሡ', 'ሢ', 'ሣ', 'ሤ', 'ሥ', 'ሦ'], 'ሠ family (sz)', (0x1220, 0x1226)),
            CharacterFamily('r', ['ረ', 'ሩ', 'ሪ', 'ራ', 'ሬ', 'ር', 'ሮ'], 'ረ family (r)', (0x1228, 0x122E)),
            CharacterFamily('s', ['ሰ', 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ'], 'ሰ family (s)', (0x1230, 0x1236)),
            CharacterFamily('sh', ['ሸ', 'ሹ', 'ሺ', 'ሻ', 'ሼ', 'ሽ', 'ሾ'], 'ሸ family (sh)', (0x1238, 0x123E)),
            CharacterFamily('q', ['ቀ', 'ቁ', 'ቂ', 'ቃ', 'ቄ', 'ቅ', 'ቆ'], 'ቀ family (q)', (0x1240, 0x1246)),
            CharacterFamily('b', ['በ', 'ቡ', 'ቢ', 'ባ', 'ቤ', 'ብ', 'ቦ'], 'በ family (b)', (0x1260, 0x1266)),
            CharacterFamily('t', ['ተ', 'ቱ', 'ቲ', 'ታ', 'ቴ', 'ት', 'ቶ'], 'ተ family (t)', (0x1270, 0x1276)),
            CharacterFamily('ch', ['ቸ', 'ቹ', 'ቺ', 'ቻ', 'ቼ', 'ች', 'ቾ'], 'ቸ family (ch)', (0x1278, 0x127E)),
            CharacterFamily('n', ['ነ', 'ኑ', 'ኒ', 'ና', 'ኔ', 'ን', 'ኖ'], 'ነ family (n)', (0x1290, 0x1296)),
            CharacterFamily('ny', ['ኘ', 'ኙ', 'ኚ', 'ኛ', 'ኜ', 'ኝ', 'ኞ'], 'ኘ family (ny)', (0x1298, 0x129E)),
            CharacterFamily('k', ['ከ', 'ኩ', 'ኪ', 'ካ', 'ኬ', 'ክ', 'ኮ'], 'ከ family (k)', (0x12A8, 0x12AE)),
            CharacterFamily('w', ['ወ', 'ዉ', 'ዊ', 'ዋ', 'ዌ', 'ው', 'ዎ'], 'ወ family (w)', (0x12C8, 0x12CE)),
            CharacterFamily('z', ['ዘ', 'ዙ', 'ዚ', 'ዛ', 'ዜ', 'ዝ', 'ዞ'], 'ዘ family (z)', (0x12D8, 0x12DE)),
            CharacterFamily('zh', ['ዠ', 'ዡ', 'ዢ', 'ዣ', 'ዤ', 'ዥ', 'ዦ'], 'ዠ family (zh)', (0x12E0, 0x12E6)),
            CharacterFamily('y', ['የ', 'ዩ', 'ዪ', 'ያ', 'ዬ', 'ይ', 'ዮ'], 'የ family (y)', (0x12E8, 0x12EE)),
            CharacterFamily('d', ['ደ', 'ዱ', 'ዲ', 'ዳ', 'ዴ', 'ድ', 'ዶ'], 'ደ family (d)', (0x12F0, 0x12F6)),
            CharacterFamily('j', ['ጀ', 'ጁ', 'ጂ', 'ጃ', 'ጄ', 'ጅ', 'ጆ'], 'ጀ family (j)', (0x1300, 0x1306)),
            CharacterFamily('g', ['ገ', 'ጉ', 'ጊ', 'ጋ', 'ጌ', 'ግ', 'ጎ'], 'ገ family (g)', (0x1308, 0x130E)),
            CharacterFamily('f', ['ፈ', 'ፉ', 'ፊ', 'ፋ', 'ፌ', 'ፍ', 'ፎ'], 'ፈ family (f)', (0x1348, 0x134E)),
            CharacterFamily('p', ['ፐ', 'ፑ', 'ፒ', 'ፓ', 'ፔ', 'ፕ', 'ፖ'], 'ፐ family (p)', (0x1350, 0x1356)),
        ]
        
        for family in default_families:
            self.add_family(family)
    
    def add_family(self, family: CharacterFamily):
        """Add a character family to the store."""
        self.families[family.base_key] = family
        # Build reverse lookup
        for char in family.characters:
            self._character_to_family[char] = family.base_key
    
    def get_family(self, base_key: str) -> Optional[CharacterFamily]:
        """Get character family by base key."""
        return self.families.get(base_key)
    
    def get_character(self, base_key: str, vowel_key: str) -> Optional[str]:
        """Get character by base consonant and vowel keys."""
        family = self.get_family(base_key)
        if family:
            return family.get_character_by_latin_key(vowel_key)
        return None
    
    def find_family_for_character(self, character: str) -> Optional[str]:
        """Find which family a character belongs to."""
        return self._character_to_family.get(character)
    
    def get_all_base_keys(self) -> List[str]:
        """Get all available base consonant keys."""
        return list(self.families.keys())
    
    def has_family(self, base_key: str) -> bool:
        """Check if a base key has a character family."""
        return base_key in self.families

In [ ]:
# Test character store
store = CharacterStore()

# Test basic functionality
test_eq(store.get_character('h', 'a'), 'ሃ')
test_eq(store.get_character('s', 'e'), 'ሴ')
test_eq(store.get_character('l', ''), 'ለ')

# Test family lookup
h_family = store.get_family('h')
test(h_family is not None)
test_eq(h_family.name, 'ሀ family (h)')

# Test reverse lookup
test_eq(store.find_family_for_character('ሃ'), 'h')
test_eq(store.find_family_for_character('ሴ'), 's')

# Test existence checks
test(store.has_family('h'))
test(not store.has_family('xyz'))

print(f"Character store contains {len(store.get_all_base_keys())} families")
print(f"Base keys: {store.get_all_base_keys()[:10]}...")  # Show first 10

## Interactive Character Exploration

Let's create some helper functions to explore character families interactively.

In [ ]:
#| export
def explore_family(store: CharacterStore, base_key: str):
    """Display detailed information about a character family."""
    family = store.get_family(base_key)
    if not family:
        print(f"No family found for key '{base_key}'")
        return
    
    print(f"\n{family.name}")
    print("=" * len(family.name))
    print(f"Base key: {family.base_key}")
    print(f"Unicode range: U+{family.unicode_range[0]:04X} - U+{family.unicode_range[1]:04X}")
    print("\nCharacters:")
    
    for i, char in enumerate(family.characters):
        vowel = VowelOrder.from_index(i)
        print(f"  {vowel.geez_name:4} ({vowel.latin_key:2}): {char} (U+{ord(char):04X})")

def show_composition_examples(store: CharacterStore, examples: List[Tuple[str, str]]):
    """Show examples of character composition."""
    print("\nComposition Examples:")
    print("====================")
    print(f"{'Input':10} {'Output':8} {'Unicode':10} {'Family':15}")
    print("-" * 50)
    
    for base_key, vowel_key in examples:
        char = store.get_character(base_key, vowel_key)
        if char:
            input_str = f"{base_key}{vowel_key}"
            unicode_str = f"U+{ord(char):04X}"
            family_name = store.get_family(base_key).name.split('(')[0].strip()
            print(f"{input_str:10} {char:8} {unicode_str:10} {family_name:15}")
        else:
            print(f"{base_key}{vowel_key:10} {'N/A':8} {'N/A':10} {'N/A':15}")

In [ ]:
# Interactive exploration examples
store = CharacterStore()

# Explore the 'h' family
explore_family(store, 'h')

In [ ]:
# Show composition examples
examples = [
    ('h', 'a'),   # ሃ
    ('s', 'e'),   # ሴ
    ('l', 'o'),   # ሎ
    ('m', 'a'),   # ማ
    ('n', 'a'),   # ና
]

show_composition_examples(store, examples)

In [ ]:
# Explore different families
for key in ['s', 'l', 'm']:
    explore_family(store, key)

## Export Functionality

Add functionality to save and load character stores for configuration.

In [ ]:
#| export
def save_character_store(store: CharacterStore, filepath: Path):
    """Save character store to JSON file."""
    data = {
        'families': {
            key: {
                'base_key': family.base_key,
                'characters': family.characters,
                'name': family.name,
                'unicode_range': family.unicode_range
            }
            for key, family in store.families.items()
        }
    }
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_character_store(filepath: Path) -> CharacterStore:
    """Load character store from JSON file."""
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    store = CharacterStore()
    store.families.clear()  # Remove default families
    store._character_to_family.clear()
    
    for key, family_data in data['families'].items():
        family = CharacterFamily(
            base_key=family_data['base_key'],
            characters=family_data['characters'],
            name=family_data['name'],
            unicode_range=tuple(family_data['unicode_range'])
        )
        store.add_family(family)
    
    return store

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()